In [ ]:
pip install transformers datasets nltk rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a2769b8030873c5265dea3cc348449a3da94f3706cfb868d22512a3b68c62524
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

In [ ]:
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from tqdm import tqdm
import nltk

# Download NLTK resources
nltk.download('punkt')

# Load T5 model and tokenizer
model_name = "t5-small"  # You can also use "t5-base" or "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the CNN/DailyMail dataset (test split)
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")

# Initialize lists to store results
inputs_list = []
references_list = []
generated_list = []
rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []
bleu_scores = []

# Define the ROUGE scorer
rouge_scorer_instance = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Smoothing function for BLEU score
smooth_fn = SmoothingFunction().method1

# Function to calculate ROUGE-1, ROUGE-2, ROUGE-L, and BLEU-1 scores
def calculate_scores(reference, generated):
    # ROUGE Scores
    rouge_scores = rouge_scorer_instance.score(reference, generated)
    rouge_1 = rouge_scores["rouge1"].fmeasure
    rouge_2 = rouge_scores["rouge2"].fmeasure
    rouge_l = rouge_scores["rougeL"].fmeasure

    # BLEU-1 Score
    reference_tokens = nltk.word_tokenize(reference)
    generated_tokens = nltk.word_tokenize(generated)
    bleu = sentence_bleu([reference_tokens], generated_tokens, weights=(1, 0, 0, 0), smoothing_function=smooth_fn)

    return rouge_1, rouge_2, rouge_l, bleu

# Generate summaries and calculate scores for 10 samples
num_samples = 10
for i in tqdm(range(num_samples)):
    input_text = "summarize: " + dataset[i]["article"]
    reference_summary = dataset[i]["highlights"]

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = inputs.to(device)

    # Generate summary
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Calculate ROUGE-1, ROUGE-2, ROUGE-L, and BLEU-1 scores
    rouge_1, rouge_2, rouge_l, bleu = calculate_scores(reference_summary, generated_summary)

    # Store the results
    inputs_list.append(dataset[i]["article"])
    references_list.append(reference_summary)
    generated_list.append(generated_summary)
    rouge_1_scores.append(rouge_1)
    rouge_2_scores.append(rouge_2)
    rouge_l_scores.append(rouge_l)
    bleu_scores.append(bleu)

# Create a DataFrame and save to CSV
results_df = pd.DataFrame({
    "Input Article": inputs_list,
    "Reference Summary": references_list,
    "Generated Summary": generated_list,
    "ROUGE-1 Score": rouge_1_scores,
    "ROUGE-2 Score": rouge_2_scores,
    "ROUGE-L Score": rouge_l_scores,
    "BLEU Score": bleu_scores
})

# Save the results to a CSV file
csv_file = "t5_cnn_dailymail_all_rouge_bleu_scores.csv"
results_df.to_csv(csv_file, index=False)

# Download the CSV file
from google.colab import files
files.download(csv_file)

print(f"Results saved and downloaded as {csv_file}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 10/10 [00:53<00:00,  5.33s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Results saved and downloaded as t5_cnn_dailymail_all_rouge_bleu_scores.csv


In [ ]:
pip install transformers datasets nltk rouge_score openpyxl


In [ ]:
import pandas as pd
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from tqdm import tqdm
import nltk
from openpyxl import load_workbook
import os

# Download NLTK resources
nltk.download('punkt')

# Load BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the CNN/DailyMail dataset (test split)
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")

# Initialize lists to store results
inputs_list = []
references_list = []
generated_list = []
rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []
bleu_1_scores = []

# Define the ROUGE scorer
rouge_scorer_instance = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Smoothing function for BLEU score
smooth_fn = SmoothingFunction().method1

# Function to calculate ROUGE-1, ROUGE-2, ROUGE-L, and BLEU-1 scores
def calculate_scores(reference, generated):
    # ROUGE Scores
    rouge_scores = rouge_scorer_instance.score(reference, generated)
    rouge_1 = rouge_scores["rouge1"].fmeasure
    rouge_2 = rouge_scores["rouge2"].fmeasure
    rouge_l = rouge_scores["rougeL"].fmeasure

    # BLEU-1 Score
    reference_tokens = nltk.word_tokenize(reference)
    generated_tokens = nltk.word_tokenize(generated)
    bleu_1 = sentence_bleu([reference_tokens], generated_tokens, weights=(1, 0, 0, 0), smoothing_function=smooth_fn)

    return rouge_1, rouge_2, rouge_l, bleu_1

# Generate summaries and calculate scores for 10 samples
num_samples = 10
for i in tqdm(range(num_samples)):
    input_text = dataset[i]["article"]
    reference_summary = dataset[i]["highlights"]

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = inputs.to(device)

    # Generate summary
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Calculate ROUGE-1, ROUGE-2, ROUGE-L, and BLEU-1 scores
    rouge_1, rouge_2, rouge_l, bleu_1 = calculate_scores(reference_summary, generated_summary)

    # Store the results
    inputs_list.append(input_text)
    references_list.append(reference_summary)
    generated_list.append(generated_summary)
    rouge_1_scores.append(rouge_1)
    rouge_2_scores.append(rouge_2)
    rouge_l_scores.append(rouge_l)
    bleu_1_scores.append(bleu_1)

# Create a DataFrame for BART results
bart_results_df = pd.DataFrame({
    "Input Article": inputs_list,
    "Reference Summary": references_list,
    "Generated Summary": generated_list,
    "ROUGE-1 Score": rouge_1_scores,
    "ROUGE-2 Score": rouge_2_scores,
    "ROUGE-L Score": rouge_l_scores,
    "BLEU-1 Score": bleu_1_scores
})

# Define the output file name
csv_file = "t5_cnn_dailymail_all_rouge_bleu_scores.xlsx"

# Check if the file already exists
if not os.path.exists(csv_file):
    # Create a new Excel file with BART results if it doesn't exist
    with pd.ExcelWriter(csv_file, engine="openpyxl") as writer:
        bart_results_df.to_excel(writer, sheet_name="BART Results", index=False)
    print(f"File {csv_file} created with BART results.")
else:
    # Append BART results to a new sheet in the existing Excel file
    with pd.ExcelWriter(csv_file, mode="a", engine="openpyxl") as writer:
        bart_results_df.to_excel(writer, sheet_name="BART Results", index=False)
    print(f"BART results have been added to a new sheet in {csv_file}.")

# Download the Excel file
from google.colab import files
files.download(csv_file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 10/10 [03:21<00:00, 20.19s/it]

File t5_cnn_dailymail_all_rouge_bleu_scores.xlsx created with BART results.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>